## Valorant Decision Tree Model

In [67]:
# Initial imports
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import accuracy_score 
from sqlalchemy import create_engine, Column, Integer, String, Boolean, Float, ForeignKey

# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# from kerastuner.tuners import RandomSearch
# from tensorflow.keras.callbacks import EarlyStopping


## Loading and Preparing Agent Data

In [68]:
import sqlite3

# Create database connection
engine = create_engine('sqlite:///SQLite/Valorant.sqlite')

In [69]:
# Querying and creating Dataframe
query_1 = """
    SELECT agents.agent, team_stats.roundsWon, round_results.roundResult, player_stats.competitiveTier, player_stats.roundsPlayed, player_stats.score, match_info.gameLengthMillis, player_stats.assists, player_stats.kills, player_stats.deaths, team_stats.numPoints
    FROM agents
    INNER JOIN round_results ON agents.id = round_results.id
    INNER JOIN player_stats ON agents.id = player_stats.id
    INNER JOIN player_kills ON agents.id = player_kills.id 
    INNER JOIN match_info ON agents.id = match_info.id
    INNER JOIN team_stats ON agents.id = team_stats.id
"""

agent_df = pd.read_sql(query_1, engine)
agent_df

,agent,roundsWon,roundResult,competitiveTier,roundsPlayed,score,gameLengthMillis,assists,kills,deaths,numPoints
0,Gekko,9,Bomb detonated,IRON_1,22,1140,2046675,0,4,8,9
1,Fade,13,Eliminated,IRON_2,22,6855,2083360,3,22,16,13
2,Breach,13,Eliminated,UNRANKED,22,3669,2155617,2,12,17,13
3,Deadlock,8,Eliminated,IRON_2,22,4817,2307563,5,17,13,8
4,Raze,8,Eliminated,IRON_2,22,2580,2453157,4,9,11,8
5,Chamber,13,Bomb detonated,IRON_3,22,2875,1866841,10,10,10,13
6,KAY/O,13,Eliminated,IRON_3,22,1723,2228659,3,4,18,13
7,Skye,9,Bomb defused,IRON_3,22,7816,1732559,4,27,12,9
8,Cypher,13,Bomb defused,IRON_1,22,3586,1916596,3,12,10,13
9,Sova,10,Eliminated,IRON_3,22,3974,1956727,3,14,16,10


In [70]:
from sklearn.preprocessing import LabelEncoder

# Initializing LabelEncoder for roundResult and competitiveTier
label_encoder1 = LabelEncoder()
label_encoder2 = LabelEncoder()
label_encoder3 = LabelEncoder()

# Applying label encoding
agent_df["roundResult_encoded"] = label_encoder1.fit_transform(agent_df["roundResult"])
agent_df["competitiveTier_encoded"] = label_encoder2.fit_transform(agent_df["competitiveTier"])
agent_df["agent_encoded"] = label_encoder3.fit_transform(agent_df["agent"])

# Dropping and rearranging columns
columns_to_drop = ["roundResult", "competitiveTier", "agent"]
agent_df_refined = agent_df.drop(columns=columns_to_drop)

agent_df_refined

,roundsWon,roundsPlayed,score,gameLengthMillis,assists,kills,deaths,numPoints,roundResult_encoded,competitiveTier_encoded,agent_encoded
0,9,22,1140,2046675,0,4,8,9,1,5,7
1,13,22,6855,2083360,3,22,16,13,2,6,6
2,13,22,3669,2155617,2,12,17,13,2,10,1
3,8,22,4817,2307563,5,17,13,8,2,6,5
4,8,22,2580,2453157,4,9,11,8,2,6,15
5,13,22,2875,1866841,10,10,10,13,1,7,3
6,13,22,1723,2228659,3,4,18,13,2,7,10
7,9,22,7816,1732559,4,27,12,9,0,7,18
8,13,22,3586,1916596,3,12,10,13,0,5,4
9,10,22,3974,1956727,3,14,16,10,2,7,19


In [71]:
# Isolating our independent variable X
X = agent_df_refined.drop("roundsWon", axis=1).values
X[:5]

array([[     22,    1140, 2046675,       0,       4,       8,       9,
              1,       5,       7],
       [     22,    6855, 2083360,       3,      22,      16,      13,
              2,       6,       6],
       [     22,    3669, 2155617,       2,      12,      17,      13,
              2,      10,       1],
       [     22,    4817, 2307563,       5,      17,      13,       8,
              2,       6,       5],
       [     22,    2580, 2453157,       4,       9,      11,       8,
              2,       6,      15]])

In [72]:
# Isolating our dependent variable y
y = agent_df_refined["roundsWon"].values
y

array([ 9, 13, 13,  8,  8, 13, 13,  9, 13, 10, 13,  6, 11, 13, 13,  6,  6,
       13,  7, 13, 13,  8, 13,  6,  4, 13,  6, 13,  9, 13, 13,  8, 13, 10,
       13,  5, 13,  8, 13,  3, 13,  6, 13,  8, 13,  9])

## Creating the Decision Tree Model

In [73]:
# Spliting data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=78)

In [74]:
# Create a StandardScaler instance
scaler = StandardScaler()

In [75]:
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)

In [76]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Fitting the Decision Tree Model

In [77]:
# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

In [78]:
# Fitting the model
model = model.fit(X_train_scaled, y_train)

## Making Predictions and Model Evaluation

In [79]:
# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

In [80]:
# Calculate the classification report
class_report = classification_report(y_test, predictions)
acc_score = accuracy_score(y_test, predictions)

# Print the classification report
print(class_report)
print(f"Accuracy Score : {acc_score}")

              precision    recall  f1-score   support

           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         0
           6       0.67      1.00      0.80         2
           7       0.00      0.00      0.00         1
           8       1.00      1.00      1.00         1
           9       0.00      0.00      0.00         1
          10       0.50      1.00      0.67         1
          13       1.00      1.00      1.00         5

    accuracy                           0.75        12
   macro avg       0.40      0.50      0.43        12
weighted avg       0.65      0.75      0.69        12

Accuracy Score : 0.75


/Users/john/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/john/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/john/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/john/opt/anaconda3/lib/